<a href="https://colab.research.google.com/github/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/main/Logic/TaskLogic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys,re,operator


try:
    # Step 1: Clone the GitHub repository if not already present
    if not os.path.exists("/content/Project-Pheonix"):
        !git clone https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix.git /content/Project-Pheonix

    # Step 2: Change directory to project root
    %cd /content/Project-Pheonix

    # Step 3: Checkout the 'main' branch (or develop if you have one)
    !git fetch origin -q
    !git checkout main -q

    # Step 4: Add project directory to Python path
    sys.path.append("/content/Project-Pheonix/Logic")

    # Step 5: Install required Python packages (quietly)
    %pip install -q firebase
    %pip install -q importnb

    print("✅ Setup completed successfully.")
    from importnb import Notebook
    with Notebook():
        import CloudDB as dbService

except Exception as e:
    print("❌ Setup failed:", str(e))

# Clear installation output for cleaner display
from IPython.display import clear_output
clear_output()

In [ ]:
def get_all_tasks():
    try:
        return dbService.get_from_db('Tasks') or {}
    except Exception as e:
        print(f"Error getting tasks: {e}")
        return {}

def add_task(task_data):
    try:
        dbService.add_task(task_data)
        return True
    except Exception as e:
        print(f"Error adding task: {e}")
        return False

def get_task(task_name):
    try:
        tasks = dbService.get_from_db('Tasks') or {}
        print(f"Debug - tasks data: {tasks}")  # Debug line
        print(f"Debug - tasks type: {type(tasks)}")  # Debug line

        # Handle Firebase's dictionary structure
        if isinstance(tasks, dict):
            for task_id, task_data in tasks.items():
                print(f"Debug - task_id: {task_id}, task_data: {task_data}, type: {type(task_data)}")  # Debug line

                # Check if task_data is a dictionary
                if isinstance(task_data, dict):
                    if task_data.get('name') == task_name:
                        return task_data
                # Handle case where task_data is a string (task name directly)
                elif isinstance(task_data, str):
                    if task_data == task_name:
                        return {'name': task_data, 'importance': 'Medium', 'value': ''}
                # Handle other data types
                else:
                    print(f"Warning - unexpected task_data type: {type(task_data)} for task_id: {task_id}")

        # Handle case where tasks is a list instead of dict
        elif isinstance(tasks, list):
            for task in tasks:
                if isinstance(task, dict) and task.get('name') == task_name:
                    return task
                elif isinstance(task, str) and task == task_name:
                    return {'name': task, 'importance': 'Medium', 'value': ''}

        return None
    except Exception as e:
        print(f"Error getting task: {e}")
        import traceback
        traceback.print_exc()  # Print full traceback for debugging
        return None

def remove_task(task_name):
    """Remove a task from the database"""
    try:
        tasks = dbService.get_from_db('Tasks') or {}
        print(f"Debug - remove_task tasks: {tasks}")  # Debug line

        # Handle Firebase's dictionary structure
        if isinstance(tasks, dict):
            task_key_to_remove = None
            for task_id, task_data in tasks.items():
                # Check if task_data is a dictionary
                if isinstance(task_data, dict) and task_data.get('name') == task_name:
                    task_key_to_remove = task_id
                    break
                # Handle case where task_data is a string
                elif isinstance(task_data, str) and task_data == task_name:
                    task_key_to_remove = task_id
                    break

            if task_key_to_remove:
                # Use the CloudDB delete_task function instead
                return dbService.delete_task(task_name)

        return False
    except Exception as e:
        print(f"Error removing task: {e}")
        import traceback
        traceback.print_exc()  # Print full traceback for debugging
        return False
